In [3]:
!pip install tensorflow

In [4]:
import nltk

In [5]:
import tensorflow

In [6]:
from nltk.corpus import gutenberg

In [7]:
import pandas as pd


In [8]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [9]:
# load the data
Data = gutenberg.raw('shakespeare-hamlet.txt')

In [10]:
# save the file
with open('hamlet.txt', 'w') as f:
    f.write(Data)

In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# lOad the dataset
with open('hamlet.txt') as file:
  text = file.read().lower()


In [14]:
# Tokenize the teext -->  creating indexes towards
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [40]:
# Create input sequence
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
input_sequences


[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [41]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [43]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [44]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [45]:
import tensorflow as tf

In [46]:
x, y = input_sequences[:,:-1], input_sequences[:,-1]

In [47]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [49]:
# defining Ealy stopping
from tensorflow.keras.callbacks import EarlyStopping

In [50]:
early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [51]:
# train our lstm model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [52]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [53]:
from re import VERBOSE
#compilt the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [55]:
# train the model
history = model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1,)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0368 - loss: 6.4081 - val_accuracy: 0.0464 - val_loss: 7.0416
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 47ms/step - accuracy: 0.0500 - loss: 6.1143 - val_accuracy: 0.0484 - val_loss: 7.1250
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0519 - loss: 5.9507 - val_accuracy: 0.0511 - val_loss: 7.1381
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0539 - loss: 5.8739 - val_accuracy: 0.0569 - val_loss: 7.2236
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0577 - loss: 5.7735 - val_accuracy: 0.0550 - val_loss: 7.3110
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0610 - loss: 5.6844 - val_accuracy: 0.0596 - val_loss: 7.3633
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 48ms/step - accuracy: 0.0662 - loss: 5.5585 - val_accuracy: 0.0581 - val_loss: 7.4006
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 31s 47ms/step - accuracy: 0.0678 - loss: 5

In [56]:

# function to predict the next word
def predict_next_word(model, tokenizer, text,max_sequence_len):
  token_list = tokenizer.texts_to_sequences([text])[0]
  if (len(token_list)) >=max_sequence_len:
    token_list = token_list[-(max_sequence_len -1):]

  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = (model.predict(token_list,verbose=0))
  predicted_word_index = np.argmax(predicted,axis=1)

  for word, index in tokenizer.word_index.items():
    if index == predicted_word_index:
       return word

  return None





In [59]:
input_text = "TO be or not to be"
print(f"INPUT TEXT: {input_text}")
max_sequence_len=model.input_shape[1]+1

next_word = predict_next_word(model, tokenizer, input_text,max_sequence_len)
print(f"Predicted Next Word: {next_word}")

INPUT TEXT: TO be or not to be
Predicted Next Word: go


In [58]:
# save the model
model.save('next_word_lstm_model.h5')

In [ ]:
# save the tokenizer 
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)